In [ ]:
!wget https://raw.githubusercontent.com/fadilmr/TeamBased3/main/seeds_dataset.csv

--2022-12-23 03:49:56--  https://raw.githubusercontent.com/fadilmr/TeamBased3/main/seeds_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9368 (9.1K) [text/plain]
Saving to: ‘seeds_dataset.csv’

seeds_dataset.csv   100%[===================>]   9.15K  --.-KB/s    in 0s      

2022-12-23 03:49:56 (90.0 MB/s) - ‘seeds_dataset.csv’ saved [9368/9368]



In [ ]:
## Isi nama dan NIM seluruh anggota kelompok di sini
#             Nama                      NIM
#   Muhammad Fadil Maulana Akbar     1301204297
#   Muhammad Rafli Ramadhan          1301200204
#   Rahusien Salim                   1301201431
#   Shafa Diva Syahira               1301200157

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree

# Load Dataset
Inisialisasi variable, masukan NIM di sini untuk random state.

Gunakan NIM salah satu anggota kelompok.



In [ ]:
filepath = "./seeds_dataset.csv"

# Isi nim anda di sini
nim = 1301204297 

Load dataset, cek konten data

Target kelas: **Category** 

In [ ]:
df = pd.read_csv(filepath, sep="\t")
df.sample(10, random_state=nim)

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len,Category
173,11.40,13.08,0.8375,5.136,2.763,5.588,5.089,3
148,12.70,13.71,0.8491,5.386,2.911,3.260,5.316,3
47,14.99,14.56,0.8883,5.570,3.377,2.958,5.175,1
36,16.20,15.27,0.8734,5.826,3.464,2.823,5.527,1
77,20.71,17.23,0.8763,6.579,3.814,4.451,6.451,2
127,17.98,15.85,0.8993,5.979,3.687,2.257,5.919,2
134,15.56,14.89,0.8823,5.776,3.408,4.972,5.847,2
16,13.99,13.83,0.9183,5.119,3.383,5.234,4.781,1
38,14.80,14.52,0.8823,5.656,3.288,3.112,5.309,1
51,15.78,14.91,0.8923,5.674,3.434,5.593,5.136,1


Training/test split **60/40**

In [ ]:
# X: semua kolom, Y: kolom terakhir (category)
x = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.6, random_state=nim)

# We perform bootstrap on the train data
# Keep test data for final accuracy testing
len(x_train), len(x_test)

(126, 84)

In [ ]:
x_train

,Area,Perimeter,Compactness,Kernel_len,Kernel_width,Assymetry,Groove_len
168,11.35,13.12,0.8291,5.176,2.668,4.337,5.132
204,12.37,13.47,0.8567,5.204,2.960,3.919,5.001
45,13.80,14.04,0.8794,5.376,3.155,1.560,4.961
83,19.57,16.74,0.8779,6.384,3.772,1.472,6.273
193,10.82,12.83,0.8256,5.180,2.630,4.853,5.089
...,...,...,...,...,...,...,...
103,19.18,16.63,0.8717,6.369,3.681,3.357,6.229
149,10.79,12.93,0.8107,5.317,2.648,5.462,5.194
179,12.21,13.47,0.8453,5.357,2.893,1.661,5.178
33,13.94,14.17,0.8728,5.585,3.150,2.124,5.012


# Random Forest
Build a Random Forest, consisting of 20 Decision Tree classifer, with the following steps:

1. Prepare Bootstrap data (choose randomly from the *training data*, use sampling with replacement)
2. Choose 2-4 random features/atributes for each tree
3. Build Decision Tree classifier using step (1) and (2)
4. Calculate OOB score within the tree only

**NOTE**
This whole exercise has been simplified.
- Tree can be created using a library, therefore you may not be able control the feature selection on node level
- Random features are only selected in the beginning of each tree.
- OOB score is only calculated within its own tree instead.
- Typically Random Forest don't use train/test split and can be validated using OOB error. We simplify this to make it easier to test.
- You are not allowed to use a Random Forest or Bagging library.


In [ ]:
np.random.seed(nim)
n_tree = 20

forest = []
bootstrap_columns = []
for i in range(n_tree):
    # 1. Bootstrapping
    # select random rows
    n_samples = x_train.shape[0]
    rows = np.random.choice(n_samples, n_samples, replace = True)
    x_bootstrap = x_train.iloc[rows]
    y_bootstrap = y_train.iloc[rows]
    
    print('idx terpilih : ', rows)

    # Out of Bag data
    oob = np.setdiff1d(np.arange(len(x_train)), rows)
    x_oob = x_train.iloc[oob]
    y_oob = y_train.iloc[oob]

    print('idx tidak terpilih : ', len(oob))

    # 2. Random feature selection'
    feature_taken = np.random.randint(2, 4)
    columns = np.random.choice(x_train.columns, feature_taken, replace = True)
    print(columns)
    bootstrap_columns.append(columns)

    # filter based on the selected columns
    x_bootstrap = x_bootstrap[columns]

    # 3. Build decision tree
    tree_model = tree.DecisionTreeClassifier()
    forest.append(tree_model.fit(x_bootstrap, y_bootstrap))
    
    # 4. Calculate oob score only within its own tree
    oob_score = tree_model.score(x_oob[columns], y_oob)
    print("OOB Score ",i , " : " , oob_score)
    print('=========================================')

idx terpilih :  [ 92  33  98  51  21 103  13  94  32 114  28   9 113  69  99  71   9  23
  41  81 112  68 121  39  74  66 108  44  60  70 107 100  69 112  10  38
  72 109 118  21  89  29  39 122   4 124 110  28  55  66  78  10  86  32
 103  93   9  57  46 120  85 117  22   0  94  34  19  36 124  72 105  76
 118  94  40  35  40 124  94  66 105  88  73  89 104  88  94  91  38 117
  37  68  49  11 125  45  89  95   1  11  95  69  91  75  59  76  33 106
  29  83 120  51  25  41  20 116  24  23  16  44 123  82  84 110 115  15]
idx tidak terpilih :  41
['Groove_len' 'Kernel_width']
OOB Score  0  :  0.8292682926829268
idx terpilih :  [ 70  44  23  24  31  96   5  60  24  91   7   8  70  55  21  26   2  49
 106 118  37 122  62   9 125  27 104  20  96  42 122  52  42  24 108  22
  37  29 109  97 103 109 108 110  82  58  50  87  52 120 125  76  43 100
  64  60  12  40  81  50  88 122  20  83  45  13  13  68   3  66 125 107
  55  50 119  44 124 106 102  45  50 102  19  67 106  62  81 124  18  34


# Prediction and Voting
Run prediction on x_test data for each tree

Also calculate accuracy per tree here

In [ ]:
all_preds = []
for i, tree_model in enumerate(forest):
    # Each tree has different column requirements
    # Make sure you use the correct columns for each tree
    x_test_filtered = x_test[bootstrap_columns[i]]

    predictions = tree_model.predict(x_test_filtered)

    # Might as well calculate accuracy per tree here...
    acc  = np.mean(predictions == y_test)
    print("acc Tree ",i , " : " , acc)
    
    # add it to all_preds for voting later
    all_preds.append(predictions)

all_preds = np.array(all_preds)

acc Tree  0  :  0.8690476190476191
acc Tree  1  :  0.9166666666666666
acc Tree  2  :  0.7023809523809523
acc Tree  3  :  0.8690476190476191
acc Tree  4  :  0.7738095238095238
acc Tree  5  :  0.8333333333333334
acc Tree  6  :  0.7976190476190477
acc Tree  7  :  0.8571428571428571
acc Tree  8  :  0.9404761904761905
acc Tree  9  :  0.7976190476190477
acc Tree  10  :  0.8928571428571429
acc Tree  11  :  0.8452380952380952
acc Tree  12  :  0.9285714285714286
acc Tree  13  :  0.7857142857142857
acc Tree  14  :  0.7261904761904762
acc Tree  15  :  0.9285714285714286
acc Tree  16  :  0.7976190476190477
acc Tree  17  :  0.6190476190476191
acc Tree  18  :  0.8214285714285714
acc Tree  19  :  0.8333333333333334


### Perform voting on the predictions from the Random Forest

In [ ]:
# Insert voting code here
voted_predictions = []
for i in range(len(all_preds[0])):
    pred = np.bincount(all_preds[:, i]).argmax()
    voted_predictions.append(pred)

# Calculate accuracy of Test data on the Forest
acc = np.mean(voted_predictions == y_test)
acc

0.9166666666666666

After you're done with all this, please check the excel file and put the answers for every column there.